라이브러리 임포트

In [14]:
!pip install koreanize-matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 42.6 MB/s eta 0:00:00


In [15]:
import pandas as pd
import numpy as np
from plotnine import *
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns
import folium
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings('ignore')

타슈 정거장 크롤링

In [22]:
# 1) 데이터 프레임 생성
data = pd.DataFrame(columns = ['스테이션명', '위치', '상태정보', '위도', '경도'])

# 2) 타슈 사이트 접속 주소 준비(URL) : https://new.tashu.or.kr/stationList.do
url = 'https://new.tashu.or.kr/stationList.do'

# 3) url 접속하여 html 가져오기
html = urlopen(url)

# 4) html 태그를 파싱(parsing)하여 변환
bsOject = BeautifulSoup(html, 'html.parser', from_encoding = 'UTF-8')

# 5)  타슈 정류장 정보가 있는 table만 가져오기
table = bsOject.find_all('table', {'class' : 'board-tp-01 stationtable'})

# 6) 상세 정보 추출
# 6-1) table 태그 내에 tr 찾기
tr = table[0].find_all('tr')

# 6-2) 첫번째 tr(테이블의 컬럼 정보) 제외
tr = tr[1:len(tr)] # 0 번째 인덱스 제외하고 다시 tr 담기

# 6-3) 타슈 정류장의 상세 정보 추출
for index_tr in range(0, len(tr)):
  td = tr[index_tr].find_all('td')

  # 장소
  station = td[0].text.split('.')[1].strip() # 1. 무역전시관입구(택시승강장)-> . -> 무역전시관입구(택시승강장)

  # 위치
  location = td[1].text

  # 상태
  condition = td[2].text

  # 좌표추출
  # 위도 latitude
  lat = td[3].button.attrs['data-lat']

  # 경도 Longitude
  lon = td[3].button.attrs['data-ltd']

  # 7) 타슈 정류장 상세 정보 dataframe 담기(append)
  data = data.append({'스테이션명' : station,
                      '위치' : location,
                      '상태정보' : condition,
                      '위도' : float(lat),
                      '경도' : float(lon)}, ignore_index= True)
  print('conplets of ' + station)

print('-----------------------------')
print(data)






conplets of 무역전시관입구(택시승강장)
conplets of 대전컨벤션센터
conplets of 한밭수목원1
conplets of 초원아파트(104동 버스정류장)
conplets of 둔산대공원 입구(버스정류장)
conplets of 백합네거리(농협)
conplets of 정부청사 입구(대덕대로)
conplets of 정부청사 입구(샘머리)
conplets of 만년동 KBS 부근(기업은행)
conplets of 누리아파트(후문 버스정류장)
conplets of 정부청사역 4번출구
conplets of 삼천중학교(정문 맞은편)
conplets of 둔산 하이마트
conplets of 둔산 홈플러스(법원 버스정류장)
conplets of 국화아파트(501동)
conplets of 타임월드
conplets of 대전시청
conplets of 현대아이텔(시청역 6번출구)
conplets of 크로바아파트(정문 맞은편)
conplets of 카이스트 학사식당
conplets of 카이스트 창의학습관
conplets of 카이스트 정문
conplets of 화폐박물관
conplets of 카이스트 후문
conplets of 엑스포과학공원 주차장
conplets of 엑스포과학공원
conplets of 스마트시티(201동)
conplets of 만년동 행정복지센터
conplets of 선사유적지공용주차장
conplets of 유성구청
conplets of 홈플러스(유성점)
conplets of 월평역 2번출구(육교밑)
conplets of 진달래아파트(107동)
conplets of 갈마역 2번출구
conplets of 가람아파트(3동)
conplets of 수정타운아파트(13동)
conplets of 갑천역 1번출구
conplets of 패션월드
conplets of 월평타운
conplets of 대전일보
conplets of 갈마네거리
conplets of 둔산여고
conplets of 을지대학병원
conplets of 청솔아파트(9동)
conplets of

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   스테이션명   260 non-null    object 
 1   위치      260 non-null    object 
 2   상태정보    260 non-null    object 
 3   위도      260 non-null    float64
 4   경도      260 non-null    float64
dtypes: float64(2), object(3)
memory usage: 10.3+ KB


In [25]:
# 지도 시각화 
geo_df = data.copy()

map = folium.Map(location = [geo_df['위도'].mean(),geo_df['경도'].mean()], zoom_start = 13)

for n in geo_df.index:
  if geo_df.loc:
      icon_color = 'green'
  folium.Marker([geo_df.loc[n,'위도'], geo_df.loc[n,'경도']],
                icon = folium.Icon(color = icon_color, icon = 'bicycle', prefix = 'fa')).add_to(map)

map

In [27]:
data.to_csv('타슈크롤링_20230418.csv',encoding = 'utf-8-sig', index = False)

크롤링한 데잍 

데이터 준비하기

In [28]:
df = pd.read_csv('/content/타슈크롤링_20230418.csv')

지도 시각화

In [32]:
# 1)  타슈 정류장 위치기반으로 중심좌표 설정
dj_map = folium.Map(location = [df['위도'].mean(), df['경도'].mean()], zoom_start=14)

# 2) 타슈 정류장 위치에 마커 올리 
for index_draw in df.index:
  folium.Marker([df.loc[index_draw]['위도'],df.loc[index_draw]['경도']],
                popup = df.loc[index_draw]['스테이션명'],
                icon = folium.Icon(icon = 'bicycle', prefix = 'fa')).add_to(dj_map)
        
dj_map
